<a href="https://colab.research.google.com/github/koominji/CrawlingWanted/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C(%EC%9A%B0%EB%8C%80%EC%82%AC%ED%95%AD)_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## "원티드"사이트에서 채용공고 중에서 "우대사항"만 크롤링한 코드입니다.

In [2]:
!pip install selenium

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 356 kB 36.7 MB/s 
     |████████████████████████████████| 138 kB 59.8 MB/s 
     |████████████████████████████████| 3.6 MB 31.9 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#  원티드 -> 탐색

driver_path='chromedriver.exe'

driver = webdriver.Chrome(executable_path=driver_path)
driver.implicitly_wait(3)
driver.get("https://www.wanted.co.kr/wdlist?country=kr&job_sort=job.latest_order&years=-1&locations=all")
driver.implicitly_wait(3)

time.sleep(0.5)


## 새창뜨면 로그인하기

In [ ]:
# 맨 밑까지 스크롤 내리는 코드

last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3.0)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_page_height == last_page_height:
        break
    last_page_height = new_page_height

In [ ]:
########## 스크롤 3번만 내리기

last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

for i in range(3):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3.0)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_page_height == last_page_height:
        break
    last_page_height = new_page_height

In [ ]:
# 내린 페이지 코드 가져오기
page = driver.page_source
soup = BeautifulSoup(page,"html.parser") #html로 파싱

# job card 긁어오기
articles=soup.find_all('div',{'class':'_3D4OeuZHyGXN7wwibRM5BJ'})


In [ ]:
# 총 글 갯수
len(articles)

10551

In [ ]:
result=[]
resultList=[]

In [ ]:
count=0

In [ ]:
for article in articles:
    
    time.sleep(2)
    
    # 회사이름
    companyName = article.find('div',{'class':'job-card-company-name'})
    cn=companyName.text
    print(cn)
    
    # 채용포지션
    jobPosition = article.find('div',{'class':'job-card-position'})
    jp=jobPosition.text
    print(jp)
    
    # 각 페이지 접속
    each_job_page='https://www.wanted.co.kr'+article.a['href']
    driver.get(each_job_page)
    
    page = driver.page_source
    soup = BeautifulSoup(page,"html.parser") #html로 파싱
    
    #print(soup)
    # 공고 제목
    #print(title.text)
    
    # 내용 전체 가져옴 -> content
    time.sleep(3)
    content=soup.find('div',{'class':'_31EtVNPZ-KwYCXvVZ3927g'})
    
    try:
        # !!! 우대사항만 뽑아오기
        c=content.text
        temp=c[c.find("우대사항")+4:c.find("혜택 및 복지")]
        
        result=[cn,jp,temp]
        resultList.append(result)
    
        result_df = pd.DataFrame(resultList,columns=['company Name','job position','Preferential matters'])
   
    except: # 예외 나는 페이지 버림
        print("")
    
    count+=1
    
    

라네띠
웹디자이너
씨디에프브로스
데이터 마케팅팀 팀장급
백패커(idus)
[아이디어스] 푸드 카테고리 작가 영입(브랜드 소싱&영업) 담당


In [ ]:
result_df.to_csv('wantedTest.csv', encoding="utf-8-sig")